In [1]:
# Import packages
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
import netCDF4 as nc

In [2]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\geospatial_internship\\datasets')
shape_telangana = gpd.read_file('telangana_shapefile.geojson')
shape_telangana = shape_telangana.explode()

C:\Users\Jesse\AppData\Local\Temp\ipykernel_12008\1639596723.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  shape_telangana = shape_telangana.explode()


In [6]:
path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\c_gls_BA300_202205310000_GLOBE_S3_V1.2.1.nc'

In [7]:
nc_metadata = nc.Dataset(path)
nc_metadata

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    processing_level: L3
    identifier: urn:cgls:global:ba300_v1_333m:BA300_202205310000_GLOBE_S3_V1.2.1
    institution: VITO NV
    time_coverage_end: 2022-05-31T23:59:59Z
    source: Derived from EO satellite imagery
    processing_mode: Near Real Time
    copyright: Copernicus Service information 2022
    references: https://land.copernicus.eu/global/products/ba
    sensor: OLCI,SLSTR
    long_name: Burnt Area
    product_version: V1.2.1
    orbit_type: LEO
    time_coverage_start: 2022-05-21T00:00:00Z
    platform: Sentinel-3
    title: 10-daily Burnt Area 333M: GLOBE 2022-05-31T00:00:00Z
    archive_facility: VITO
    parent_identifier: urn:cgls:global:ba300_v1_333m
    history: 2022-06-03: Processing line BA
    dimensions(sizes): lon(120960), lat(47040), time(1)
    variables(dimensions): uint8 BA_DEKAD(time, lat, lon), uint16 FDOB_DEKAD(time, lat, lon), uint16 FDOB_SE

In [11]:
# Import the netCDF data for Leaf Area Index
nc_file = gdal.Open('NETCDF:c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\c_gls_BA300_202205310000_GLOBE_S3_V1.2.1.nc:NOBS')
# Convert the netCDF to Geotiff file
tif_file = gdal.Translate('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif', nc_file)

In [13]:
# In the next two code blocks, we crop the Geotiff file to Burnt Area in Telangana
ba_data = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif"
with rasterio.open(ba_data) as src:
    out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
    out_meta = src.meta

In [14]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save the Geotiff file to Burnt Area in Telangana
with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_cropped_tiff.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    

In [13]:
# For each of the BA images, we crop the Geotiff file to BA in Telangana
folder_names = os.listdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input')
for i in folder_names:
    file_names = os.listdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input\\' + i)
    for j in file_names:
        BA_data = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input\\" + i + "/" + j
        with rasterio.open(BA_data) as src:
            out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
            out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        # photometric = 'YCbCr'
        # photometric = 'RGB', compress = 'JPEG'
        # Save the Geotiff file to BA in Telangana
        with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Output\\" + j, "w", **out_meta) as dest:
            dest.write(out_image)

In [ ]:
#for %N in (C:/Users/Jesse/OneDrive/Documenten/Output/*.tiff) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Output/%~nN.tiff  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tiff -co COMPRESS=LZW -co TILED=YES